## 回帰モデルのバリエーション 

### リッジ回帰  
#### 係数が大きくなりすぎないよう制約を加えた回帰モデル過学習防止につながる 正規化項として「係数の2乗の合計」を用いる
バイアスが高い：予測結果自体は密集しているが、根本的な予測結果の平均値が100から遠くへ離れている  
    →対処方法：モデルを複雑にして、訓練データの様々な法則をモデルに学習させる　※弊害として学習になりえる
バリアンスが高い：予測結果の平均は100に近いが予測結果自体にばらつきがおおきく生じている。※モデルが複雑なら予測結果のばらつきが大きくなる
    →対処方法：データの件数を増やす、同じような学習結果になる分析手法を選択する
    
実際と予測の誤差 = バイアス＋バリアンス＋ノイズ（外れ値）

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("sukkiri-ml2-codes/datafiles/Boston.csv")
df_str = df[["CRIME"]]
df_dummy = pd.get_dummies(df_str, drop_first=True) #文字列データを数値化する
df_without = df.drop(df_str.columns ,axis=1)
df2 = pd.concat([df_without, df_dummy], axis=1)

df2 =df2.fillna(df2.mean())

df2 = df2.drop([76]) #外れ値削除

t=df2[["PRICE"]]
x=df2.loc[:,["RM","PTRATIO","LSTAT"]]

sc = StandardScaler()#データの標準化
sc_x = sc.fit_transform(x)
sc2 = StandardScaler()
sc_t = sc2.fit_transform(t)

from sklearn.preprocessing import PolynomialFeatures #予測精度を上げるために必要な多項式公や交互作用特徴量を作成するツール
pf = PolynomialFeatures(degree=2, include_bias=False)
pf_x = pf.fit_transform(sc_x)
pf.get_feature_names_out() #カラム番号で管理されているので使いづらい

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

### 線形回帰で過学習が起きる事を確認

In [110]:
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test =train_test_split(pf_x, sc_t, test_size=0.3, random_state=0)

model = LinearRegression()
model.fit(x_train,y_train)

print(model.score(x_train, y_train))
model.score(x_test,y_test)

0.8763373117732821


0.5490731214664921

### リッジ回帰で過学習が起こるか確認

In [113]:
from sklearn.linear_model import Ridge

ridmodel = Ridge(alpha=10)#alpha=定数　適正な数値で過学習防止、数値が小さすぎると通常の回帰分析に、大きすぎると予測性能が悪くなる
ridmodel.fit(x_train, y_train)

print(ridmodel.score(x_train, y_train))
print(ridmodel.score(x_test, y_test))

0.8658617097578485
0.5907242365074143


### ラッソ回帰  
#### 予測にあまり役に立たないような特徴量を学習中に削除する　 正規化項として「係数の絶対値の合計」を用いる

In [116]:
from sklearn.linear_model import Lasso

x_train, x_test, y_train, y_test =train_test_split(pf_x, sc_t, test_size=0.3, random_state=0)

model = Lasso(alpha=0.1)
model.fit(x_train,y_train)

print(model.score(x_train, y_train))
model.score(x_test,y_test)

weg = model.coef_
pd.Series(weg,index=pf.get_feature_names_out())

0.819163876203935


x0       0.409953
x1      -0.108089
x2      -0.244670
x0^2     0.151187
x0 x1   -0.000000
x0 x2   -0.021443
x1^2    -0.000000
x1 x2    0.000000
x2^2     0.000000
dtype: float64

### 回帰木  
#### 回帰のためのフローチャート

In [130]:
df2.head()

x = df2.loc[:,"ZN":"LSTAT"]
t = df2["PRICE"]
x_train, x_test, y_train, y_test =train_test_split(x, t, test_size=0.3, random_state=0)

from sklearn.tree import DecisionTreeRegressor #回帰木バージョン

#木の深さの最大を10
model = DecisionTreeRegressor(max_depth=10, random_state=0)
model.fit(x_train,y_train)
model.score(x_test, y_test)

pd.Series(model.feature_importances_, index=x.columns) #特徴量の重要度を参照

ZN         0.000008
INDUS      0.003588
CHAS       0.000000
NOX        0.028132
RM         0.766687
AGE        0.016723
DIS        0.003451
RAD        0.009045
TAX        0.012735
PTRATIO    0.008856
LSTAT      0.150775
dtype: float64

## 誤差と過学習
- 未知のデータにおける予測と実測値の誤差は、バイアスとバリアンスとノイズに分解することができる
- ノイズは正解データの分散
- バリアンスは予測結果の分散
- バイアスは正解データの平均値と予測結果の平均値の誤差
- 過学習は、バリアンスが高い状態のこと


### ロジスティック回帰実装

In [19]:
df = pd.read_csv("sukkiri-ml2-codes/datafiles/iris.csv")
df.head()

,がく片長さ,がく片幅,花弁長さ,花弁幅,種類
0,0.22,0.63,0.08,0.04,Iris-setosa
1,0.17,0.42,0.35,0.04,Iris-setosa
2,0.11,0.50,0.13,0.04,Iris-setosa
3,0.08,0.46,0.26,0.04,Iris-setosa
4,0.19,0.67,0.44,0.04,Iris-setosa
